# Download Dependencies

In [1]:
%AddDeps com.lucidworks.spark spark-solr 3.6.0 --transitive
%AddDeps edu.stanford.nlp stanford-corenlp 3.9.2 --transitive
%AddDeps edu.stanford.nlp stanford-corenlp 3.9.2 --classifier models-english
%AddDeps edu.stanford.nlp stanford-corenlp 3.9.2 --classifier models-english-kbp

Marking com.lucidworks.spark:spark-solr:3.6.0 for download
-> Failed to resolve org.restlet.jee:org.restlet:2.3.0
    -> not found: /tmp/toree-tmp-dir5098819600856165967/toree_add_deps/cache/org.restlet.jee/org.restlet/ivy-2.3.0.xml
    -> not found: https://repo1.maven.org/maven2/org/restlet/jee/org.restlet/2.3.0/org.restlet-2.3.0.pom
-> Failed to resolve org.restlet.jee:org.restlet.ext.servlet:2.3.0
    -> not found: /tmp/toree-tmp-dir5098819600856165967/toree_add_deps/cache/org.restlet.jee/org.restlet.ext.servlet/ivy-2.3.0.xml
    -> not found: https://repo1.maven.org/maven2/org/restlet/jee/org.restlet.ext.servlet/2.3.0/org.restlet.ext.servlet-2.3.0.pom
Obtained 388 files
Marking edu.stanford.nlp:stanford-corenlp:3.9.2 for download
Obtained 40 files
Marking edu.stanford.nlp:stanford-corenlp:3.9.2 for download
Obtained 1 files
Marking edu.stanford.nlp:stanford-corenlp:3.9.2 for download
Obtained 1 files


# Fetch Documents from Solr

In [5]:
import com.lucidworks.spark.rdd.SelectSolrRDD

val SOLR = "localhost:9990"

val INDEX = "core17"
val FIELD = "contents"
val QUERY = "Obama"

val rdd = new SelectSolrRDD(SOLR, INDEX, sc)
    .rows(1000)
    .query(FIELD + ":" + QUERY)
    .count()

SOLR = localhost:9990
INDEX = core17
FIELD = contents
QUERY = Obama
rdd = 899


899

# CoreNLP Example

In [ ]:
import collection.JavaConversions._
import edu.stanford.nlp.simple._

val doc = new Document("Barrack Obama was born in Hawaii. He is our president.")

for (sent <- doc.sentences()) {
    for (triple <- sent.openieTriples()) {
        println(s"(${triple.subjectLemmaGloss()}, ${triple.relationLemmaGloss()}, ${triple.objectLemmaGloss()})")
    }
}

In [ ]:
%LsMagic

# Solr + Spark + CoreNLP

In [ ]:
import collection.mutable.ListBuffer
import collection.JavaConversions._

import com.lucidworks.spark.rdd.SelectSolrRDD
import edu.stanford.nlp.simple.Document

val SOLR = "localhost:9990"

val INDEX = "core17"
val FIELD = "contents"
val QUERY = "Obama"

val rdd = new SelectSolrRDD(SOLR, INDEX, sc)
    .rows(1000)
    .query(FIELD + ":" + QUERY)
    .map(d => {
        
        val list = new ListBuffer[Tuple3[String, String, String]]()
        val doc = new Document(d.get("raw").asInstanceOf[String])
        
        for (sent <- doc.sentences()) {
            for (triple <- sent.openieTriples()) {
                list.append((triple.subjectLemmaGloss(), triple.relationLemmaGloss(), triple.objectLemmaGloss()))
            }
        }
        
        (d.get("id"), list.toList)
        
    })

val doc = rdd.take(1)

println(doc.head._1)
doc.head._2.foreach(println)